In [38]:
import ts.auth as a
import os
from datetime import datetime, time, timedelta
import pytz
import pandas as pd
from pandas.tseries.offsets import QuarterEnd, Week
import json
from dotenv import load_dotenv
load_dotenv()

key = os.getenv('TRADESTATION_KEY')
secret = os.getenv('TRADESTATION_SECRET')
redirect = os.getenv('TRADESTATION_REDIRECT_URI')

def get_session_date(timestamp):
    # Convert timestamp to EST
    est = pytz.timezone('US/Eastern')
    ts_est = timestamp.astimezone(est)

    # Define session start and end times
    session_start = time(18, 0)  # 6 PM
    session_end = time(17, 1)    # 5 PM

    # If the time is between 6 PM and midnight, it belongs to the next day's session
    if ts_est.time() >= session_start:
        return ts_est.date() + timedelta(days=1)
    # If the time is between midnight and 5 PM, it belongs to the current day's session
    elif ts_est.time() < session_end:
        return ts_est.date()
    # If the time is between 5 PM and 6 PM, it belongs to the previous day's session
    else:
        return ts_est.date()

def get_current_contract(date):
    # Convert input to pandas Timestamp if it's not already
    date = pd.Timestamp(date)

    # Find the next quarter end
    next_quarter_end = date + QuarterEnd(0)

    # Calculate the third Friday of the quarter's last month
    third_friday = next_quarter_end - Week(weekday=4)
    while third_friday.day < 15:
        third_friday += Week()

    # If we're past the third Friday of the current quarter's last month,
    # move to the next quarter
    if date > third_friday:
        next_quarter_end += QuarterEnd(1)
        third_friday = next_quarter_end - Week(weekday=4)
        while third_friday.day < 15:
            third_friday += Week()

    # Determine the contract month code
    month_code = {
        3: 'H',
        6: 'M',
        9: 'U',
        12: 'Z'
    }

    contract_month = next_quarter_end.month
    contract_year = next_quarter_end.year % 100  # Get last two digits of year

    # Construct the contract code
    contract_code = f"{month_code[contract_month]}{contract_year}"

    #   return {
    #       'input_date': date,
    #       'contract_code': contract_code,
    #       'contract_month': next_quarter_end.strftime('%b %Y'),
    #       'last_trading_day': third_friday
    #   }
    return contract_code


def add_session_flags(df):
    # Ensure the index is in datetime format and in Eastern Time
    df.index = pd.to_datetime(df.index, utc=True).tz_convert('America/New_York')
    
    # Create time objects for session boundaries
    asian_start = pd.Timestamp('19:00').time()
    asian_end = pd.Timestamp('04:00').time()
    european_start = pd.Timestamp('03:00').time()
    european_end = pd.Timestamp('12:00').time()
    na_start = pd.Timestamp('08:00').time()
    na_end = pd.Timestamp('17:00').time()
    
    # Create boolean columns for each session
    df['ASIA'] = ((df.index.time >= asian_start) | (df.index.time < asian_end))
    df['EU'] = ((df.index.time >= european_start) & (df.index.time < european_end))
    df['NA'] = ((df.index.time >= na_start) & (df.index.time < na_end))
    
    return df

client = a.easy_client(key, secret, redirect, paper_trade=False)

contract = get_current_contract(datetime.now())

symbol_contract = f'@NQ{contract}'

params = {'symbol': symbol_contract
    , 'interval': 5
    , 'unit': 'Minute'
    , 'barsback': 1440
    # , firstdate = None
    , 'lastdate': datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ")
    , 'sessiontemplate': 'Default'}

resp = client.get_bars(**params)

if resp.is_success:
    
    data = pd.json_normalize(json.loads(resp.content)['Bars'])
    data['TimeStamp'] = pd.to_datetime(data['TimeStamp'], utc=True)
    data['SessionDate'] = data['TimeStamp'].apply(get_session_date)

    data['Delta'] = data.groupby('SessionDate')['TimeStamp'].diff().bfill()
    data['EndBar'] = data['TimeStamp'].dt.tz_convert('America/New_York')
    data['Datetime'] = data['EndBar'] - data['Delta']
    data['ActiveContract'] = symbol_contract.replace('@','')
    data['Open'] = data['Open'].astype(float)
    data['High'] = data['High'].astype(float)
    data['Low'] = data['Low'].astype(float)
    data['Close'] = data['Close'].astype(float)
    data['TotalVolume'] = data['TotalVolume'].astype(float)

    final_data = data[['Datetime','Open','High','Low','Close','TotalVolume','ActiveContract','SessionDate']]
    final_data.columns = ['Datetime','Open','High','Low','Close','Volume','ActiveContract','SessionDate']
    final_data = final_data.set_index('Datetime')

    # Create features
    final_data['ClosePts'] = final_data['Close'].diff()
    final_data['ClosePct'] = final_data['ClosePts']/final_data['Close']
    final_data = add_session_flags(final_data)
    print(final_data.tail())
    
else:
    print(resp)

                               Open      High       Low     Close  Volume  \
Datetime                                                                    
2024-08-26 02:55:00-04:00  19788.25  19792.50  19785.75  19787.75   177.0   
2024-08-26 03:00:00-04:00  19788.25  19796.50  19782.25  19791.00   435.0   
2024-08-26 03:05:00-04:00  19791.50  19806.75  19790.50  19804.50   528.0   
2024-08-26 03:10:00-04:00  19805.50  19810.50  19798.75  19807.25   402.0   
2024-08-26 03:15:00-04:00  19808.50  19811.75  19808.25  19811.00    80.0   

                          ActiveContract  ClosePts  ClosePct  ASIA     EU  \
Datetime                                                                    
2024-08-26 02:55:00-04:00          NQU24      0.00  0.000000  True  False   
2024-08-26 03:00:00-04:00          NQU24      3.25  0.000164  True   True   
2024-08-26 03:05:00-04:00          NQU24     13.50  0.000682  True   True   
2024-08-26 03:10:00-04:00          NQU24      2.75  0.000139  True   True  

In [39]:
final_data.groupby(['ASIA','EU','NA'])['Volume'].median()

ASIA   EU     NA   
False  False  False     219.5
              True     3068.5
       True   False     385.5
              True     4590.0
True   False  False     203.0
       True   False     369.0
Name: Volume, dtype: float64